# Darcy equation: exercise 3

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = 0\\
\nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \qquad p = 1 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega$$
The matrix permeability is defined in the following way
$$
k(x, y) = 
\left\{
\begin{array}{ll}
k_1 & 0.2 < y < 0.4\\
k_2 & 0.6 < y < 0.8\\
1 & \text{otherwise}
\end{array}
\right.
$$
with, for example, $k_1 = k_2 = 10^{-2}$. Compare the effective permeability computed analytically or numerically.

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules.

In [15]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, to facilitate the imposizione of $k$ we consider a structured grid from PorePy and then convert it into a PyGeoN grid.

In [16]:
N, dim = 10, 2
# creation of the grid
sd = pp.StructuredTriangleGrid([N] * dim, [1] * dim)
# compute the geometrical properties of the grid
sd.compute_geometry()

With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [17]:
key = "flow"
k1, k2 = 1e-2, 1e-2
bc_val = []
bc_ess = []

# declare the discretization objects, useful to setup the data
rt0 = pg.RT0(key)
p0 = pg.PwConstants(key)

# set up the data for the flow problem
data = {}

# heterogeneous permeability tensor
y = sd.cell_centers[1, :]

perm_vals = np.ones(sd.num_cells)
perm_vals[np.logical_and(y > 0.2, y < 0.4)] = k1
perm_vals[np.logical_and(y > 0.6, y < 0.8)] = k2

perm = pp.SecondOrderTensor(perm_vals)
parameters = {
    "second_order_tensor": perm,
}
pp.initialize_data(sd, data, key, parameters)

# with the following steps we identify the portions of the boundary
# to impose the boundary conditions
left_right = np.logical_or(sd.face_centers[0, :] == 0, sd.face_centers[0, :] == 1)
top_bottom = np.logical_or(sd.face_centers[1, :] == 0, sd.face_centers[1, :] == 1)
ess_p_dofs = np.zeros(p0.ndof(sd), dtype=bool)


# compute the pressure boundary condition, which is a natural condition for the RT0 space
def p_bc(x):
    return 1 - x[1]


bc_val = -rt0.assemble_nat_bc(sd, p_bc, top_bottom)
bc_ess = np.hstack((left_right, ess_p_dofs))

Once the data are assigned to the grid, we construct the matrices. In particular, the linear system associated with the equation is given as
$$
\left(
\begin{array}{cc} 
M & -B^\top\\
B & 0
\end{array}
\right)
\left(
\begin{array}{c} 
q\\ 
p
\end{array}
\right)
=\left(
\begin{array}{c} 
p_{\partial}\\ 
0
\end{array}
\right)
$$

In [18]:
# construct the local matrices
mass_rt0 = rt0.assemble_mass_matrix(sd, data)
mass_p0 = p0.assemble_mass_matrix(sd, data)
div = mass_p0 @ rt0.assemble_diff_matrix(sd)

# assemble the saddle point problem
spp = sps.bmat([[mass_rt0, -div.T], [div, None]], format="csc")

# get the degrees of freedom for each variable
dof_p, dof_q = div.shape

# assemble the right-hand side
rhs = np.zeros(dof_p + dof_q)
rhs[:dof_q] += bc_val

We solve the linear system and extract the two solutions $q$ and $p$.

In [19]:
# solve the problem
ls = pg.LinearSystem(spp, rhs)
ls.flag_ess_bc(bc_ess, np.zeros(dof_q + dof_p))
x = ls.solve()

# extract the variables
q = x[:dof_q]
p = x[-dof_p:]

Since the computed $q$ is one value per facet of the grid, for visualization purposes we project the flux in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [20]:
# post process variables
proj_q = rt0.eval_at_cell_centers(sd)
cell_q = (proj_q * q).reshape((3, -1), order="F")
cell_p = p0.eval_at_cell_centers(sd) * p

save = pp.Exporter(sd, "sol", folder_name="ex3")
save.write_vtu([("cell_p", cell_p), ("cell_q", cell_q), ("permeability", perm_vals)])

Let us compute now the effective permeability, analytically we can use the following expression
$$
k_{\perp}^{eff} = \frac{5}{\frac{3}{k_0} + \frac{1}{k_1} + \frac{1}{k_2}}
$$
while, by considering the Darcy law we can approximate numerically the permeability as
$$
 q = - k \nabla p \quad \Rightarrow \quad q \cdot \nu|_{bottom} = - \tilde{k}_{\perp}^{eff} \frac{p_{top} - p_{bottom}}{\Delta y} 
 \quad \Rightarrow \quad \tilde{k}_{\perp}^{eff} = \frac{q \cdot \nu|_{bottom} \Delta y}{p_{top} -p_{bottom}}
$$
by considering the geometry and boundary conditions of the current problem then we obtain
$$
\tilde{k}_{\perp}^{eff} = q \cdot \nu|_{bottom}.
$$

In [21]:
# compute the numerical effective permeability
bottom = sd.face_centers[1, :] == 0
normal = sd.face_normals[:, bottom] / sd.face_areas[bottom]
perm_eff_num = np.sum(q[bottom] * normal)

# compute the analytical effective permeability
perm_eff_ana = 5 / (3 + 1 / k1 + 1 / k2)

print("Numerical effective permeability", perm_eff_num)
print("Analytical effective permeability", perm_eff_ana)
print("Relative error", np.abs(perm_eff_ana - perm_eff_num) / perm_eff_ana)

Numerical effective permeability 0.024630541871921183
Analytical effective permeability 0.024630541871921183
Relative error 0.0
